<a href="https://colab.research.google.com/github/jhuarancca/ASU_DataProcessing/blob/main/assigment04_RangeQuery.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#!/usr/bin/python3
#
# Assignment2 Interface
#
import psycopg2
#import assigment03


outputRangeFile = 'RangeQueryOut.txt'
outputPointFile = 'PointQueryOut.txt'

def getOpenConnection(user='postgres', password='1234', dbname='postgres'):
    conn = None
    try:
        # read connection parameters
        # params = config()

        # connect to the postgresql server
        print('connecting to the postgresql database...')
        # conn = psycopg2.connect(**params)
        conn = psycopg2.connect(
            "dbname='" + dbname + "' user='" + user + "' host='localhost' password='" + password + "'")

        # create a cursor
        cur = conn.cursor()

        # execute a statement
        print('postgresql database version:')
        cur.execute('select version()')

        # display the postgresql database server version
        db_version = cur.fetchone()
        print(db_version)

        # close the communication with the postgresql
        cur.close()
        return conn
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        # finally:
        #    if conn is not None:
        #        conn.close()
        #        print('Database connection closed.')


# Donot close the connection inside this file i.e. do not perform openconnection.close()
def RangeQuery(ratingsTableName, ratingMinValue, ratingMaxValue, openconnection):
    result= []

    with openconnection.cursor() as cursor:
        lsSql = "SELECT table_name FROM information_schema.tables WHERE table_schema = 'public' and table_name <>  'ratings' "
        cursor.execute(lsSql)

        tables = cursor.fetchall()

        for table in tables:
            print(table[0])
            if not (table[0] == 'rangeratingsmetadata' or table[0] == 'roundrobinratingsmetadata'):
                sqlSelectCommand = "SELECT * FROM {} WHERE Rating >= {} AND Rating <= {}".format(table[0] , ratingMinValue, ratingMaxValue)
                cursor.execute(sqlSelectCommand)
                values = cursor.fetchall()

                for each_value in values:
                    newRow = "{},{},{},{}".format(table[0], each_value[0], each_value[1], each_value[2])
                    print( each_value[0], each_value[1], each_value[2])
                    print(newRow)
                    result.append(newRow)
        print(result)
        writeToFile(outputRangeFile, result)
        cursor.close()
        print('Range query completed :'+outputRangeFile)


def PointQuery(ratingsTableName, ratingValue, openconnection):
    result = []

    with openconnection.cursor() as cursor:
        sqlSelectCommand = "SELECT table_name FROM information_schema.tables WHERE table_schema = 'public'  and table_name <>  'ratings' "
        cursor.execute(sqlSelectCommand)
        tables = cursor.fetchall()

        for eachTable in tables:
            if not (eachTable[0] == 'rangeratingsmetadata' or eachTable[0] == 'roundrobinratingsmetadata'):
                sqlSelectCommand = "SELECT * FROM {} WHERE Rating = {}".format(eachTable[0], ratingValue)
                cursor.execute(sqlSelectCommand)
                values = cursor.fetchall()

                for eachValue in values:
                    newRow = "{},{},{},{}".format(eachTable[0], eachValue[0], eachValue[1], eachValue[2])
                    result.append(newRow)

        writeToFile(outputPointFile, result)
        cursor.close()
        print('Range query completed :'+outputRangeFile)


def writeToFile(filename, rows):
    f = open(filename, 'w')
    for line in rows:
        f.write(line)
        f.write('\n')
    f.close()

#from assigment03 import createDB
#from assigment03 import loadRatings
#from assigment03 import rangePartition
#from assigment03 import roundRobinPartition

if __name__ == '__main__':
    print('Hello')
    conn = getOpenConnection()
    #createDB()
    #loadRatings('ratings', 'test_data.txt', conn)
    #rangePartition('ratings', 2, conn)
    #roundRobinPartition('ratings', 10, conn)

    RangeQuery('ratings',1,3,conn)
    PointQuery('ratings',4,conn)

Hello
connecting to the postgresql database...
could not connect to server: Connection refused
	Is the server running on host "localhost" (127.0.0.1) and accepting
	TCP/IP connections on port 5432?
could not connect to server: Cannot assign requested address
	Is the server running on host "localhost" (::1) and accepting
	TCP/IP connections on port 5432?



AttributeError: ignored